In [43]:
# !pip install langchain-openai
# !pip install pinecone
# !pip install pandas
# !pip install langchain-pinecone

---
llm으로 json 설명 추가

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

data_path = './../../datas' 
file_path = os.path.join(data_path, 'flower_data.json')
output_file_path = os.path.join(data_path, 'flower_data_completed.json')

In [3]:
import json

with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

In [4]:
input_datas = [plant['flowNm'] for plant in data]
input_datas

['향나무',
 '나도풍란',
 '군자란',
 '삼나무',
 '수선화',
 '파키라',
 '스토크',
 '아나나스',
 '아마릴리스',
 '우묵사스레피',
 '앵초',
 '브리세아',
 '치자나무',
 '카틀레야',
 '온시디움',
 '크로커스',
 '벤자민고무나무',
 '팔손이',
 '아프리칸 바이올렛',
 '우단일엽',
 '파초',
 '호랑가시나무',
 '사스레피나무',
 '굴거리나무',
 '산호수',
 '꽝꽝나무',
 '구상나무',
 '프리뮬러',
 '괭이눈',
 '칼세올라리아',
 '네펜데스',
 '남천',
 '냉이',
 '동백나무',
 '코르딜리네',
 '디펜바키아',
 '레몬',
 '무스카리',
 '산자고',
 '스노우드롭',
 '스위트피',
 '복수초',
 '산거울',
 '솜나물',
 '아잘레아',
 '안스리움',
 '골담초',
 '에크메아',
 '제라늄',
 '끈끈이주걱',
 '필레아',
 '너도밤나무',
 '탱자나무',
 '프리지아',
 '고란초',
 '석위',
 '렉스베고니아',
 '깽깽이풀',
 '솜다리',
 '우산나물',
 '갯버들',
 '고사리',
 '잉글리쉬데이지',
 '버드나무',
 '벚나무',
 '산수유',
 '서향',
 '시네라리아',
 '히어리',
 '유채',
 '남산제비꽃',
 '제비꽃',
 '조팝나무',
 '진달래',
 '양지꽃',
 '튜립',
 '노루발풀',
 '솜방망이',
 '팬지',
 '황매화',
 '히아신스',
 '피나물',
 '삼지닥나무',
 '노루귀',
 '미선나무',
 '할미꽃',
 '얼레지',
 '각시붓꽃',
 '바람꽃',
 '현호색',
 '앉은부채',
 '개나리',
 '꽃베고니아',
 '라넌큘러스',
 '라일락',
 '애기나리',
 '금낭화',
 '매실나무',
 '천남성',
 '목련',
 '물망초',
 '박태기나무',
 '배나무',
 '명자꽃',
 '꽃댕강나무',
 '살구',
 '수양버들',
 '아디안텀',
 '아이리스',
 '월계수',
 '으름',
 '자작나무',
 '전나무(젓나무)',
 '봄맞이꽃',
 '

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    ### 입력 데이터 ###
    {input_data}

    ### 설명 ###
    당신은 식물 전문가입니다. 입력 데이터의 식물에 대해 다음 4가지 정보를 생성하여 반환하세요.
    - 설명 문장이나 불릿 없이, JSON만 출력하세요.
    - 반드시 항목명(flowNm, desc, difficulty, interior, style)을 그대로 사용하세요.

    0. flowNm: 식물 이름
    1. desc: 꽃의 특징과 키우는 방법을 요약한 소개글 (100자 이내)
    2. difficulty: 키우는 난이도 ('매우 쉬움', '쉬움', '보통', '어려움', '매우 어려움' 중 하나)
    3. interior: 이 식물로 공간을 연출하는 구체적인 방법
    4. style: 이 식물과 가장 잘 어울리는 인테리어 스타일

    ### 출력 형식 ###
    {{
        "flowNm": "",
        "desc": "",
        "difficulty": "",
        "interior": "",
        "style": ""
    }}
    """,
    input_variables=["input_data"],
)

model = ChatOpenAI(
		model='gpt-4o-mini',
		temperature=1
)

chain = prompt | model

In [6]:
input_datas = ['튤립', '사과나무']

In [7]:
completed_list = []

for idx, item in enumerate(input_datas):
    raw = chain.invoke({'input_data': item})
    
    try:
        obj = json.loads(raw.content)
    except json.JSONDecodeError:
        print(f"{idx}번째 입력에서 JSON 파싱 실패: {item}")
        print("raw:", raw)
        continue

    completed_list.append(obj)

c:\Users\Playdata\AppData\Local\anaconda3\envs\project3\lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


In [8]:
completed_list

[{'flowNm': '튤립',
  'desc': '튤립은 다채로운 색상과 우아한 형태를 가진 꽃으로, 햇빛을 좋아하며 물빠짐이 좋은 토양에서 잘 자랍니다.',
  'difficulty': '쉬움',
  'interior': '튤립을 화분에 심어 거실 창가에 두거나, 식탁 중앙에 화려한 꽃다발로 배열하여 공간을 밝힙니다.',
  'style': '모던'},
 {'flowNm': '사과나무',
  'desc': '사과나무는 아름다운 꽃과 과일을 탐스럽게 맺으며, 햇볕과 물을 적절히 주면 잘 자랍니다.',
  'difficulty': '보통',
  'interior': '사과나무를 실내 또는 정원에 배치해 자연스러운 아늑한 느낌을 주며, 화분을 활용해 레이어드 배치 가능합니다.',
  'style': '내추럴 인테리어'}]

In [9]:
import json

with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(completed_list, f, ensure_ascii=False, indent=2)

---
벡터DB

In [10]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)

In [11]:
index_name = 'plants'

pc.delete_index(index_name)

In [12]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [13]:
import pandas as pd

df = pd.read_json('./../../datas/flower_data.json')

In [14]:
def create_text_for_embedding(row):
    air_cond_text = '공기정화식물이다.' if row['isAirCond'] else ''
    toxic_dog_text = '강아지에게 해롭다.' if row['isToxicToDog'] else ''
    toxic_cat_text = '고양이에게 해롭다.' if row['isToxicToCat'] else ''

    return (
        f"꽃 이름: {row['flowNm']}, "
        f"꽃말: {row['flowLang']}, "
        f"특징: {row['fContent']}, "
        f"용도: {row['fUse']}, "
        f"재배법: {row['fGrow']}, "
        f"종류: {row['fType']}. "
        f"{air_cond_text} "
        f"{toxic_dog_text} "
        f"{toxic_cat_text}"
    ).strip()

df['text_to_embed'] = df.apply(create_text_for_embedding, axis=1)

texts = df['text_to_embed'].tolist()

In [15]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
embeddings = embedding_model.embed_documents(texts)

idx = pc.Index(index_name)

for i, row in df.iterrows():
    metadatas = row.drop(['fSctNm', 'fEngNm', 'fileName1', 'fileName2', 'fileName3', 'publishOrg', 'colors', 'text_to_embed']).to_dict()
    metadatas["text"] = row["text_to_embed"]

    idx.upsert([
        (
            str(row['dataNo']),
            embeddings[i],
            metadatas
        )
    ])

c:\Users\Playdata\AppData\Local\anaconda3\envs\project3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
collected_data = {
    "purpose": "거실 / 거실의 빈 벽이 너무 심심해서 놓고 싶어",
    "preferred_style": "미니멀",
    "preferred_color": "#A1B07C",
    "plant_type": '난',
    "season": "여름",
    "humidity": "건조",
    "has_dog": True,
    "has_cat": False,
    "isAirCond": True,
    "watering_frequency": "주1회 가능",
    "user_experience": "초보",
    "emotion": "위로",
}

prev_text = ['튤립', '사랑초']

---
RAG

In [18]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

query_text = str(collected_data)

retrievals = retriever.batch([query_text])

In [19]:
retrievals[0]

[Document(id='343', metadata={'dataNo': 343.0, 'fContent': '꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다.', 'fDay': 8.0, 'fGrow': '5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온 사무실 같은 곳에서도 잘 자란다. 다육성 식물이지만 물도 좋아하며 번식은 엽삽이나 포기나누기로 잘 증식된다.', 'fMonth': 12.0, 'fType': "돌나물과에 속하는 온실 관화식물이다. 꽃색이 다양한 품종들이 개발되어 있으며 꽃이 초롱같은 '앤디'라는 품종도 흔히 판매된다.", 'fUse': '다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다.', 'flowLang': '인기, 설레임', 'flowNm': '칼랑코에', 'imgUrl1': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-1.jpg', 'imgUrl2': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-2.jpg', 'imgUrl3': 'https://www.nihhs.go.kr/Attfiles/FLOWF/1208-3.jpg', 'isAirCond': True, 'isToxicToCat': False, 'isToxicToDog': False}, page_content="꽃 이름: 칼랑코에, 꽃말: 인기, 설레임, 특징: 꽃모양도 다양하고 꽃색도 여러가지다. 꽃도 홓꽃과 겹꽃이 있으며 겨울철 분화용으로 좋은 식물이다. 개화기간도 길고 화분이 작아 청소년들 선물용으로도 좋은 아이템이다., 용도: 다육성 분화라서 건조에도 강하므로 실내에서도 잘 견딘다. 광이 부족하면 웃자라기 때문에 볕이 잘 드는 베란다나 창가 등에 둔다., 재배법: 5도 이상이면 무난히 월동하므로 겨울철 야간에 무가온

---
RAG 기반 추천 챗봇

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

def build_recommendation(collected_data, rag_result, prev_text):
    prompt = PromptTemplate(
        template= """
            ### 요구사항 ###
            {collected_data}

            ### RAG 검색 결과(참고) ###
            {rag_result}

            ### 이미 추천한 식물 (다시 추천하지 않습니다) ###
            {prev_text}

            ### 설명 ###
            당신은 친절한 식물 전문가입니다. 한국어로 친절하게 답변하세요.
            RAG 검색 결과를 참고해서 사용자에게 식물 1가지를 추천하세요.
            이미 추천한 식물은 사용자가 거부한 식물이니 추천하지 않습니다.

            ### 응답목록 ###
            - 추천하는 식물 이름
            - 추천하는 이유
            - 식물의 꽃말
            - 식물을 키우는 법
            - 식물의 특징

            ### 출력형식 ###
            {{
                "flowNm": "",
                "response": ""
            }}
        """,
        input_variables=['collected_data', 'rag_result', 'prev_text']
)
        
    model = ChatOpenAI(
		model='gpt-4o-mini',
		temperature=1
    )
        
    chain = prompt | model

    raw = chain.invoke({'collected_data': collected_data, 'rag_result': rag_result, 'prev_text': prev_text})
    response = json.loads(raw.content)
    
    return response

In [40]:
collected_data = {
    "purpose": "거실 / 거실의 빈 벽이 너무 심심해서 놓고 싶어",
    "preferred_style": "미니멀",
    "preferred_color": "#A1B07C",
    "plant_type": '난',
    "season": "여름",
    "humidity": "건조",
    "has_dog": True,
    "has_cat": False,
    "isAirCond": True,
    "watering_frequency": "주1회 가능",
    "user_experience": "초보",
    "emotion": "위로",
    "description":"설명~",
}

prev_text = ['튤립', '사랑초']

In [41]:
res = build_recommendation(collected_data, retrievals, prev_text)

In [42]:
res

{'flowNm': '칼랑코에',
 'response': '칼랑코에는 미니멀한 스타일의 인테리어에 잘 어울리는 다육성 식물입니다. 건조한 환경에서도 잘 자라며, 주1회의 물 주기로 관리하실 수 있습니다. 이 식물은 꽃모양과 색상이 다양해 거실의 빈 벽을 화사하게 꾸며줄 것입니다.',
 'flowerLanguage': '인기, 설레임',
 'careInstructions': '칼랑코에는 햇빛을 좋아하지만 너무 강한 직사광선은 피해야 합니다. 배수가 잘 되는 화분에 심고, 주1회 물을 주시되 흙이 완전히 마른 후에 물을 주는 것이 좋습니다.',
 'features': '다양한 꽃 모양과 색상이 있으며, 개화기간이 길고 다육성이어서 생존력이 뛰어납니다. 또한, 실내에서도 잘 자라는 특성을 가지고 있어 초보자에게 적합합니다.'}